In [7]:
import pdfplumber

def extract_text_from_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
    return text

pdf_text = extract_text_from_pdf("mock_trial.pdf")


In [8]:
def estimate_tokens(text):
    return len(text) // 4

total_tokens = estimate_tokens(pdf_text)
print(f"Estimated tokens: {total_tokens}")


Estimated tokens: 5215


In [9]:
judge_prompt = f"""
You are a judge presiding over a mock trial. You must adhere to the trial's context and answer questions as the judge.

Here is the trial document:
{pdf_text} 

You are ready to proceed. The user will be roleplaying as another character in the mock trial. 
"""


In [10]:
car_owner_prompt = f"""
You are pretending to be the car owner in this mock trial. You must adhere to the trial's context and answer questions as the car owner.

Here is the trial document:
{pdf_text} 

You are ready to proceed. The user will be roleplaying as another character in the mock trial. 
"""

In [15]:
import autogen
from autogen import ConversableAgent


llm_config = {"config_list": [{"model": "gpt-4o-mini", "api_key": os.environ.get("OPENAI_API_KEY")}]}
    
# the main entrypoint/supervisor agent
judge_agent = ConversableAgent("judge agent", system_message=judge_prompt, llm_config=llm_config)

In [16]:
car_owner_agent = ConversableAgent(name="car owner agent", system_message=car_owner_prompt, llm_config=llm_config)

In [17]:
human_proxy = ConversableAgent(
    "human_proxy",
    llm_config=False,  # no LLM used for human proxy
    human_input_mode="ALWAYS",  # always ask for human input
)

In [18]:
from autogen import GroupChat
from autogen import GroupChatManager
import os

group_chat = GroupChat(
    agents=[judge_agent, car_owner_agent, human_proxy],
    messages=[],
    max_round=6,
)


group_chat_manager = GroupChatManager(
    groupchat=group_chat,
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

In [19]:
chat_result = human_proxy.initiate_chat(
    group_chat_manager,
    message="I will be roleplaying as the Deputy DA in this mock trial, is everyone ready?",
    summary_method="reflection_with_llm",
)

human_proxy (to chat_manager):

I will be roleplaying as the Deputy DA in this mock trial, is everyone ready?

--------------------------------------------------------------------------------


RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for gpt-4 in organization org-Ic0hC0Z5lo6ZQKyAPTz2eKkB on tokens per min (TPM): Limit 10000, Requested 10734. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [ ]:
# from openai import OpenAI
# client = OpenAI()

# completion = client.chat.completions.create(
#     model="gpt-4o-mini",  # Choose model with sufficient token capacity
#     messages=[{"role": "system", "content": roleplay_prompt}]
# )

# print(completion.choices[0].message)


ChatCompletionMessage(content='Understood. I am prepared to answer questions within the context of this mock trial. Please proceed with your inquiries.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
